In [ ]:
import os 
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
##Fetch Data
pathname="../input/vggface-using-tripletloss/celebs/celebs"
#pathname="../input/vggface-using-tripletloss/crop_faces/faces"
dirList=os.listdir(pathname)
print(dirList)
ln=len(dirList)
print(ln)
pathname+="/"

In [ ]:
person=0
pairs=[]
pairsy=[]

In [ ]:
##random data generator


#false pairs
for x in range(ln):
    p1=x
    p1l=os.listdir(pathname+dirList[p1])
    #for i in range(len(p1l)):
    for i in range(50):
        while True:
            p2 = random.randint(0,ln-1)
            if(p2==p1):
                continue
            else:
                break
        #print(p2)
        p2l=os.listdir(pathname+dirList[p2])
        p1p=random.randint(0,len(p1l)-1)
        #p1p=i
        p2p=random.randint(0,len(p2l)-1)
        faceImg1=cv2.imread(pathname+dirList[p1]+"/"+p1l[p1p])
        faceImg1=cv2.resize(faceImg1, (224, 224) ,interpolation = cv2.INTER_NEAREST)
        faceImg2=cv2.imread(pathname+dirList[p2]+"/"+p2l[p2p])
        faceImg2=cv2.resize(faceImg2, (224, 224), interpolation = cv2.INTER_NEAREST)
        #plt.imshow(faceImg1)
        #plt.show()
        #plt.imshow(faceImg2)
        #plt.show()
        pairs.append([faceImg1,faceImg2])
        pairsy.append(0)

del(p1l)
del(p2l)

In [ ]:
i=20
plt.imshow(pairs[i][0])
plt.show()
plt.imshow(pairs[i][1])
plt.show()

In [ ]:
#positive pair
for x in range(ln):
    p1=x
    p1l=os.listdir(pathname+dirList[p1])
    #for i in range(len(p1l)):
    for i in range(50):   
        p1p=random.randint(0,len(p1l)-1)
        #p1p=i
        f1=cv2.imread(pathname+dirList[p1]+"/"+p1l[p1p])
        f1=cv2.resize(f1, (224, 224) ,interpolation = cv2.INTER_NEAREST)


        p1p=random.randint(0,len(p1l)-1)
        f2=cv2.imread(pathname+dirList[p1]+"/"+p1l[p1p])
        f2=cv2.resize(f2, (224, 224) ,interpolation = cv2.INTER_NEAREST)


        pairs.append([f1,f2])
        pairsy.append(1)
del(p1l)

In [ ]:
i=-100
plt.imshow(pairs[i][0])
plt.show()
plt.imshow(pairs[i][1])
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
pairs, X_test, pairsy, y_test = train_test_split(pairs, pairsy, test_size=0.30)

In [ ]:
ten1=[]
ten2=[]
for i in X_test:
    ten1.append(i[0])
    ten2.append(i[1])
ten1=np.array(ten1)
ten2=np.array(ten2)

In [ ]:
del(pairs)
del(y_test)
del(X_test)

In [ ]:
from tensorflow.keras.applications import VGG16,EfficientNetB2
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,Lambda, Conv2DTranspose, Flatten,Dense,Subtract,Input

In [ ]:
#siamese nural net
model=Sequential()
model.add(EfficientNetB2(include_top=False,input_shape=(224,224,3)))
model.add(Flatten())
#model.summary()


input_shape = (224,224,3)
left_input = Input(input_shape)
right_input = Input(input_shape)


encoded_l = model(left_input)
encoded_r = model(right_input)


subtracted = Subtract()([encoded_l,encoded_r])
both = Lambda(lambda x: abs(x))(subtracted)
prediction = Dense(2,activation='softmax')(both)

siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
siamese_net.summary()

In [ ]:
import tensorflow
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
siamese_net.compile(optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy'],)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
siamese_net.fit(x=[ten1,ten2],y=np.array(pairsy),epochs=10)

In [ ]:
del(ten1)
del(ten2)
del(pairsy)

In [ ]:
print(siamese_net.predict(x=[np.array([imgp[1]]),np.array([imga[0]])]))

In [ ]:
siamese_net.save("mdl-7.h5")

In [ ]:
from tensorflow import keras
model = keras.models.load_model('../input/face-identification/mdl-6-2.h5')
